In [ ]:
# only for taurus
import os
print(os.getcwd())
os.chdir("notebooks")
print(os.getcwd())

In [ ]:
# Connect to the PostgreSQL database with psycopg2
import psycopg2

# Connection parameters
host=
database=
user=
password=
port=

# Create a connection
conn = psycopg2.connect(
    host=host,
    database=database,
    user=user,
    password=password,
    port=port
)

# Create a cursor object
cur = conn.cursor()

In [ ]:
# example exploration

# show attribute names of mimiciii.heightweight
cur.execute("SELECT column_name FROM information_schema.columns WHERE table_name = 'heightweight';")
rows = cur.fetchall()
print(rows)

# show head of mimiciii.heightweight
cur.execute("SELECT icustay_id, weight_first, height_first FROM mimiciii.heightweight LIMIT 1;")
rows = cur.fetchall()
print(rows)


In [ ]:
conn.rollback()

In [ ]:
# extract unit types

# Define the list of itemids for which to extract unit types and labels
itemids = [
    50868, 50862, 51144, 50882, 50885, 50912, 50902, 50806, 50931, 50809,
    51221, 50810, 51222, 50811, 50813, 51265, 50971, 50822, 51275, 51237,
    51274, 50983, 50824, 51006, 51301, 51300
]

# Convert the list of itemids into a string format suitable for SQL query
itemids_str = ','.join(map(str, itemids))

# Execute the SQL query to select itemid, valueuom, and label for the specified itemids
# This query prioritizes rows with a non-null valueuom for each itemid
cur.execute(f"""
SELECT MAX(le.valueuom) AS valueuom, dl.label
FROM mimiciii.labevents le
JOIN mimiciii.d_labitems dl ON le.itemid = dl.itemid
WHERE le.itemid IN ({itemids_str})
GROUP BY le.itemid, dl.label
HAVING MAX(le.valueuom) IS NOT NULL;
""")

rows = cur.fetchall()

# Print column names
column_names = [desc[0] for desc in cur.description]
print(column_names)

# Print rows to see the label and unit type for each unique measure
print(rows)

In [ ]:
# Define the list of itemids for which to extract unit types and labels
itemids = [
    50868, 50862, 51144, 50882, 50885, 50912, 50902, 50806, 50931, 50809,
    51221, 50810, 51222, 50811, 50813, 51265, 50971, 50822, 51275, 51237,
    51274, 50983, 50824, 51006, 51301, 51300, 50920, 50970, 51007, 50893,
    211, 220045, 51, 442, 455, 6701, 220179, 220050, 8368, 8440, 8441, 8555,
    220180, 220051, 456, 52, 6702, 443, 220052, 220181, 225312, 615, 618,
    220210, 224690, 223761, 678, 223762, 676, 646, 220277, 807, 811, 1529,
    3745, 3744, 225664, 220621, 226537
]

# Convert the list of itemids into a string format suitable for SQL query
itemids_str = ','.join(map(str, itemids))

# Execute the SQL query to select itemid, valueuom, and label for the specified itemids
# This query prioritizes rows with a non-null valueuom for each itemid
cur.execute(f"""
SELECT MAX(le.valueuom) AS valueuom, dl.label
FROM mimiciii.labevents le
JOIN mimiciii.d_labitems dl ON le.itemid = dl.itemid
WHERE le.itemid IN ({itemids_str})
GROUP BY le.itemid, dl.label
HAVING MAX(le.valueuom) IS NOT NULL;
""")

rows = cur.fetchall()

# Print column names
column_names = [desc[0] for desc in cur.description]
print(column_names)

# Print rows to see the label and unit type for each unique measure
print(rows)

In [ ]:
labs_extended = """
SELECT
  pvt.subject_id, pvt.hadm_id, pvt.icustay_id, pvt.charttime

  , min(CASE WHEN label = 'ANION GAP' THEN valuenum ELSE null END) as ANIONGAP_min
  , max(CASE WHEN label = 'ANION GAP' THEN valuenum ELSE null END) as ANIONGAP_max
  , min(CASE WHEN label = 'ALBUMIN' THEN valuenum ELSE null END) as ALBUMIN_min
  , max(CASE WHEN label = 'ALBUMIN' THEN valuenum ELSE null END) as ALBUMIN_max
  , min(CASE WHEN label = 'BANDS' THEN valuenum ELSE null END) as BANDS_min
  , max(CASE WHEN label = 'BANDS' THEN valuenum ELSE null END) as BANDS_max
  , min(CASE WHEN label = 'BICARBONATE' THEN valuenum ELSE null END) as BICARBONATE_min
  , max(CASE WHEN label = 'BICARBONATE' THEN valuenum ELSE null END) as BICARBONATE_max
  , min(CASE WHEN label = 'BILIRUBIN' THEN valuenum ELSE null END) as BILIRUBIN_min
  , max(CASE WHEN label = 'BILIRUBIN' THEN valuenum ELSE null END) as BILIRUBIN_max
  , min(CASE WHEN label = 'CREATININE' THEN valuenum ELSE null END) as CREATININE_min
  , max(CASE WHEN label = 'CREATININE' THEN valuenum ELSE null END) as CREATININE_max
  , min(CASE WHEN label = 'CHLORIDE' THEN valuenum ELSE null END) as CHLORIDE_min
  , max(CASE WHEN label = 'CHLORIDE' THEN valuenum ELSE null END) as CHLORIDE_max
  , min(CASE WHEN label = 'GLUCOSE' THEN valuenum ELSE null END) as GLUCOSE_min
  , max(CASE WHEN label = 'GLUCOSE' THEN valuenum ELSE null END) as GLUCOSE_max
  , min(CASE WHEN label = 'HEMATOCRIT' THEN valuenum ELSE null END) as HEMATOCRIT_min
  , max(CASE WHEN label = 'HEMATOCRIT' THEN valuenum ELSE null END) as HEMATOCRIT_max
  , min(CASE WHEN label = 'HEMOGLOBIN' THEN valuenum ELSE null END) as HEMOGLOBIN_min
  , max(CASE WHEN label = 'HEMOGLOBIN' THEN valuenum ELSE null END) as HEMOGLOBIN_max
  , min(CASE WHEN label = 'LACTATE' THEN valuenum ELSE null END) as LACTATE_min
  , max(CASE WHEN label = 'LACTATE' THEN valuenum ELSE null END) as LACTATE_max
  , min(CASE WHEN label = 'PLATELET' THEN valuenum ELSE null END) as PLATELET_min
  , max(CASE WHEN label = 'PLATELET' THEN valuenum ELSE null END) as PLATELET_max
  , min(CASE WHEN label = 'POTASSIUM' THEN valuenum ELSE null END) as POTASSIUM_min
  , max(CASE WHEN label = 'POTASSIUM' THEN valuenum ELSE null END) as POTASSIUM_max
  , min(CASE WHEN label = 'PTT' THEN valuenum ELSE null END) as PTT_min
  , max(CASE WHEN label = 'PTT' THEN valuenum ELSE null END) as PTT_max
  , min(CASE WHEN label = 'INR' THEN valuenum ELSE null END) as INR_min
  , max(CASE WHEN label = 'INR' THEN valuenum ELSE null END) as INR_max
  , min(CASE WHEN label = 'PT' THEN valuenum ELSE null END) as PT_min
  , max(CASE WHEN label = 'PT' THEN valuenum ELSE null END) as PT_max
  , min(CASE WHEN label = 'SODIUM' THEN valuenum ELSE null END) as SODIUM_min
  , max(CASE WHEN label = 'SODIUM' THEN valuenum ELSE null end) as SODIUM_max
  , min(CASE WHEN label = 'BUN' THEN valuenum ELSE null end) as BUN_min
  , max(CASE WHEN label = 'BUN' THEN valuenum ELSE null end) as BUN_max
  , min(CASE WHEN label = 'WBC' THEN valuenum ELSE null end) as WBC_min
  , max(CASE WHEN label = 'WBC' THEN valuenum ELSE null end) as WBC_max
  , min(CASE WHEN label = 'GFR' THEN valuenum ELSE null end) as GFR_min
  , max(CASE WHEN label = 'GFR' THEN valuenum ELSE null end) as GFR_max
  , min(CASE WHEN label = 'PHOSPHATE' THEN valuenum ELSE null end) as PHOSPHATE_min
  , max(CASE WHEN label = 'PHOSPHATE' THEN valuenum ELSE null end) as PHOSPHATE_max  
  , min(CASE WHEN label = 'URIC ACID' THEN valuenum ELSE null end) as URIC ACID_min
  , max(CASE WHEN label = 'URIC ACID' THEN valuenum ELSE null end) as URIC ACID_max
  , min(CASE WHEN label = 'CALCIUM' THEN valuenum ELSE null end) as CALCIUM_min
  , max(CASE WHEN label = 'CALCIUM' THEN valuenum ELSE null end) as CALCIUM_max

FROM
( -- begin query that extracts the data
  SELECT ie.subject_id, ie.hadm_id, ie.icustay_id, charttime
  -- here we assign labels to ITEMIDs
  -- this also fuses together multiple ITEMIDs containing the same data
  , CASE
        WHEN itemid = 50868 THEN 'ANION GAP'
        WHEN itemid = 50862 THEN 'ALBUMIN'
        WHEN itemid = 51144 THEN 'BANDS'
        WHEN itemid = 50882 THEN 'BICARBONATE'
        WHEN itemid = 50885 THEN 'BILIRUBIN'
        WHEN itemid = 50912 THEN 'CREATININE'
        WHEN itemid = 50806 THEN 'CHLORIDE'
        WHEN itemid = 50902 THEN 'CHLORIDE'
        WHEN itemid = 50809 THEN 'GLUCOSE'
        WHEN itemid = 50931 THEN 'GLUCOSE'
        WHEN itemid = 50810 THEN 'HEMATOCRIT'
        WHEN itemid = 51221 THEN 'HEMATOCRIT'
        WHEN itemid = 50811 THEN 'HEMOGLOBIN'
        WHEN itemid = 51222 THEN 'HEMOGLOBIN'
        WHEN itemid = 50813 THEN 'LACTATE'
        WHEN itemid = 51265 THEN 'PLATELET'
        WHEN itemid = 50822 THEN 'POTASSIUM'
        WHEN itemid = 50971 THEN 'POTASSIUM'
        WHEN itemid = 51275 THEN 'PTT'
        WHEN itemid = 51237 THEN 'INR'
        WHEN itemid = 51274 THEN 'PT'
        WHEN itemid = 50824 THEN 'SODIUM'
        WHEN itemid = 50983 THEN 'SODIUM'
        WHEN itemid = 51006 THEN 'BUN'
        WHEN itemid = 51300 THEN 'WBC'
        WHEN itemid = 51301 THEN 'WBC'
        WHEN itemid = 50920 THEN 'GFR'
        WHEN itemid = 50970 THEN 'PHOSPHATE'
        WHEN itemid = 51007 THEN 'URIC ACID'
        WHEN itemid = 50893 THEN 'CALCIUM'

      ELSE null
    END AS label
  , -- add in some sanity checks on the values
  -- the where clause below requires all valuenum to be > 0, so these are only upper limit checks
    CASE
      WHEN itemid = 50862 and valuenum >    10 THEN null -- g/dL 'ALBUMIN'
      WHEN itemid = 50868 and valuenum > 10000 THEN null -- mEq/L 'ANION GAP'
      WHEN itemid = 51144 and valuenum <     0 THEN null -- immature band forms, %
      WHEN itemid = 51144 and valuenum >   100 THEN null -- immature band forms, %
      WHEN itemid = 50882 and valuenum > 10000 THEN null -- mEq/L 'BICARBONATE'
      WHEN itemid = 50885 and valuenum >   150 THEN null -- mg/dL 'BILIRUBIN'
      WHEN itemid = 50806 and valuenum > 10000 THEN null -- mEq/L 'CHLORIDE'
      WHEN itemid = 50902 and valuenum > 10000 THEN null -- mEq/L 'CHLORIDE'
      WHEN itemid = 50912 and valuenum >   150 THEN null -- mg/dL 'CREATININE'
      WHEN itemid = 50809 and valuenum > 10000 THEN null -- mg/dL 'GLUCOSE'
      WHEN itemid = 50931 and valuenum > 10000 THEN null -- mg/dL 'GLUCOSE'
      WHEN itemid = 50810 and valuenum >   100 THEN null -- % 'HEMATOCRIT'
      WHEN itemid = 51221 and valuenum >   100 THEN null -- % 'HEMATOCRIT'
      WHEN itemid = 50811 and valuenum >    50 THEN null -- g/dL 'HEMOGLOBIN'
      WHEN itemid = 51222 and valuenum >    50 THEN null -- g/dL 'HEMOGLOBIN'
      WHEN itemid = 50813 and valuenum >    50 THEN null -- mmol/L 'LACTATE'
      WHEN itemid = 51265 and valuenum > 10000 THEN null -- K/uL 'PLATELET'
      WHEN itemid = 50822 and valuenum >    30 THEN null -- mEq/L 'POTASSIUM'
      WHEN itemid = 50971 and valuenum >    30 THEN null -- mEq/L 'POTASSIUM'
      WHEN itemid = 51275 and valuenum >   150 THEN null -- sec 'PTT'
      WHEN itemid = 51237 and valuenum >    50 THEN null -- 'INR'
      WHEN itemid = 51274 and valuenum >   150 THEN null -- sec 'PT'
      WHEN itemid = 50824 and valuenum >   200 THEN null -- mEq/L == mmol/L 'SODIUM'
      WHEN itemid = 50983 and valuenum >   200 THEN null -- mEq/L == mmol/L 'SODIUM'
      WHEN itemid = 51006 and valuenum >   300 THEN null -- 'BUN'
      WHEN itemid = 51300 and valuenum >  1000 THEN null -- 'WBC'
      WHEN itemid = 51301 and valuenum >  1000 THEN null -- 'WBC'
    ELSE le.valuenum
    END AS valuenum

  FROM mimiciii.icustays ie

  LEFT JOIN mimiciii.labevents le
    ON le.subject_id = ie.subject_id AND le.hadm_id = ie.hadm_id
    AND le.ITEMID in
    (
      -- comment is: LABEL | CATEGORY | FLUID | NUMBER OF ROWS IN LABEVENTS
      50868, -- ANION GAP | CHEMISTRY | BLOOD | 769895
      50862, -- ALBUMIN | CHEMISTRY | BLOOD | 146697
      51144, -- BANDS - hematology
      50882, -- BICARBONATE | CHEMISTRY | BLOOD | 780733
      50885, -- BILIRUBIN, TOTAL | CHEMISTRY | BLOOD | 238277
      50912, -- CREATININE | CHEMISTRY | BLOOD | 797476
      50902, -- CHLORIDE | CHEMISTRY | BLOOD | 795568
      50806, -- CHLORIDE, WHOLE BLOOD | BLOOD GAS | BLOOD | 48187
      50931, -- GLUCOSE | CHEMISTRY | BLOOD | 748981
      50809, -- GLUCOSE | BLOOD GAS | BLOOD | 196734
      51221, -- HEMATOCRIT | HEMATOLOGY | BLOOD | 881846
      50810, -- HEMATOCRIT, CALCULATED | BLOOD GAS | BLOOD | 89715
      51222, -- HEMOGLOBIN | HEMATOLOGY | BLOOD | 752523
      50811, -- HEMOGLOBIN | BLOOD GAS | BLOOD | 89712
      50813, -- LACTATE | BLOOD GAS | BLOOD | 187124
      51265, -- PLATELET COUNT | HEMATOLOGY | BLOOD | 778444
      50971, -- POTASSIUM | CHEMISTRY | BLOOD | 845825
      50822, -- POTASSIUM, WHOLE BLOOD | BLOOD GAS | BLOOD | 192946
      51275, -- PTT | HEMATOLOGY | BLOOD | 474937
      51237, -- INR(PT) | HEMATOLOGY | BLOOD | 471183
      51274, -- PT | HEMATOLOGY | BLOOD | 469090
      50983, -- SODIUM | CHEMISTRY | BLOOD | 808489
      50824, -- SODIUM, WHOLE BLOOD | BLOOD GAS | BLOOD | 71503
      51006, -- UREA NITROGEN | CHEMISTRY | BLOOD | 791925
      51301, -- WHITE BLOOD CELLS | HEMATOLOGY | BLOOD | 753301
      51300  -- WBC COUNT | HEMATOLOGY | BLOOD | 2371
    )
    AND valuenum IS NOT null AND valuenum > 0 -- lab values cannot be 0 and cannot be negative
) pvt
GROUP BY pvt.subject_id, pvt.hadm_id, pvt.icustay_id, charttime
"""

In [ ]:
labs_extended= """ SELECT pvt.subject_id, pvt.hadm_id, pvt.icustay_id, pvt.charttime, min(CASE WHEN label = 'ANION GAP' THEN valuenum ELSE null END) as ANIONGAP_min, max(CASE WHEN label = 'ANION GAP' THEN valuenum ELSE null END) as ANIONGAP_max, min(CASE WHEN label = 'ALBUMIN' THEN valuenum ELSE null END) as ALBUMIN_min, max(CASE WHEN label = 'ALBUMIN' THEN valuenum ELSE null END) as ALBUMIN_max, min(CASE WHEN label = 'BANDS' THEN valuenum ELSE null END) as BANDS_min, max(CASE WHEN label = 'BANDS' THEN valuenum ELSE null END) as BANDS_max, min(CASE WHEN label = 'BICARBONATE' THEN valuenum ELSE null END) as BICARBONATE_min, max(CASE WHEN label = 'BICARBONATE' THEN valuenum ELSE null END) as BICARBONATE_max, min(CASE WHEN label = 'BILIRUBIN' THEN valuenum ELSE null END) as BILIRUBIN_min, max(CASE WHEN label = 'BILIRUBIN' THEN valuenum ELSE null END) as BILIRUBIN_max, min(CASE WHEN label = 'CREATININE' THEN valuenum ELSE null END) as CREATININE_min, max(CASE WHEN label = 'CREATININE' THEN valuenum ELSE null END) as CREATININE_max, min(CASE WHEN label = 'CHLORIDE' THEN valuenum ELSE null END) as CHLORIDE_min, max(CASE WHEN label = 'CHLORIDE' THEN valuenum ELSE null END) as CHLORIDE_max, min(CASE WHEN label = 'GLUCOSE' THEN valuenum ELSE null END) as GLUCOSE_min, max(CASE WHEN label = 'GLUCOSE' THEN valuenum ELSE null END) as GLUCOSE_max, min(CASE WHEN label = 'HEMATOCRIT' THEN valuenum ELSE null END) as HEMATOCRIT_min, max(CASE WHEN label = 'HEMATOCRIT' THEN valuenum ELSE null END) as HEMATOCRIT_max, min(CASE WHEN label = 'HEMOGLOBIN' THEN valuenum ELSE null END) as HEMOGLOBIN_min, max(CASE WHEN label = 'HEMOGLOBIN' THEN valuenum ELSE null END) as HEMOGLOBIN_max, min(CASE WHEN label = 'LACTATE' THEN valuenum ELSE null END) as LACTATE_min, max(CASE WHEN label = 'LACTATE' THEN valuenum ELSE null END) as LACTATE_max, min(CASE WHEN label = 'PLATELET' THEN valuenum ELSE null END) as PLATELET_min, max(CASE WHEN label = 'PLATELET' THEN valuenum ELSE null END) as PLATELET_max, min(CASE WHEN label = 'POTASSIUM' THEN valuenum ELSE null END) as POTASSIUM_min, max(CASE WHEN label = 'POTASSIUM' THEN valuenum ELSE null END) as POTASSIUM_max, min(CASE WHEN label = 'PTT' THEN valuenum ELSE null END) as PTT_min, max(CASE WHEN label = 'PTT' THEN valuenum ELSE null END) as PTT_max, min(CASE WHEN label = 'INR' THEN valuenum ELSE null END) as INR_min, max(CASE WHEN label = 'INR' THEN valuenum ELSE null END) as INR_max, min(CASE WHEN label = 'PT' THEN valuenum ELSE null END) as PT_min, max(CASE WHEN label = 'PT' THEN valuenum ELSE null END) as PT_max, min(CASE WHEN label = 'SODIUM' THEN valuenum ELSE null END) as SODIUM_min, max(CASE WHEN label = 'SODIUM' THEN valuenum ELSE null END) as SODIUM_max, min(CASE WHEN label = 'BUN' THEN valuenum ELSE null END) as BUN_min, max(CASE WHEN label = 'BUN' THEN valuenum ELSE null END) as BUN_max, min(CASE WHEN label = 'WBC' THEN valuenum ELSE null END) as WBC_min, max(CASE WHEN label = 'WBC' THEN valuenum ELSE null END) as WBC_max, min(CASE WHEN label = 'GFR' THEN valuenum ELSE null END) as GFR_min, max(CASE WHEN label = 'GFR' THEN valuenum ELSE null END) as GFR_max, min(CASE WHEN label = 'PHOSPHATE' THEN valuenum ELSE null END) as PHOSPHATE_min, max(CASE WHEN label = 'PHOSPHATE' THEN valuenum ELSE null END) as PHOSPHATE_max, min(CASE WHEN label = 'URIC ACID' THEN valuenum ELSE null END) as URIC_ACID_min, max(CASE WHEN label = 'URIC ACID' THEN valuenum ELSE null END) as URIC_ACID_max, min(CASE WHEN label = 'CALCIUM' THEN valuenum ELSE null END) as CALCIUM_min, max(CASE WHEN label = 'CALCIUM' THEN valuenum ELSE null END) as CALCIUM_max FROM ( SELECT ie.subject_id, ie.hadm_id, ie.icustay_id, charttime, CASE WHEN itemid = 50868 THEN 'ANION GAP' WHEN itemid = 50862 THEN 'ALBUMIN' WHEN itemid = 51144 THEN 'BANDS' WHEN itemid = 50882 THEN 'BICARBONATE' WHEN itemid = 50885 THEN 'BILIRUBIN' WHEN itemid = 50912 THEN 'CREATININE' WHEN itemid = 50806 THEN 'CHLORIDE' WHEN itemid = 50902 THEN 'CHLORIDE' WHEN itemid = 50809 THEN 'GLUCOSE' WHEN itemid = 50931 THEN 'GLUCOSE' WHEN itemid = 50810 THEN 'HEMATOCRIT' WHEN itemid = 51221 THEN 'HEMATOCRIT' WHEN itemid = 50811 THEN 'HEMOGLOBIN' WHEN itemid = 51222 THEN 'HEMOGLOBIN' WHEN itemid = 50813 THEN 'LACTATE' WHEN itemid = 51265 THEN 'PLATELET' WHEN itemid = 50822 THEN 'POTASSIUM' WHEN itemid = 50971 THEN 'POTASSIUM' WHEN itemid = 51275 THEN 'PTT' WHEN itemid = 51237 THEN 'INR' WHEN itemid = 51274 THEN 'PT' WHEN itemid = 50824 THEN 'SODIUM' WHEN itemid = 50983 THEN 'SODIUM' WHEN itemid = 51006 THEN 'BUN' WHEN itemid = 51300 THEN 'WBC' WHEN itemid = 51301 THEN 'WBC' WHEN itemid = 50920 THEN 'GFR' WHEN itemid = 50970 THEN 'PHOSPHATE' WHEN itemid = 51007 THEN 'URIC ACID' WHEN itemid = 50893 THEN 'CALCIUM' ELSE null END AS label, CASE WHEN itemid = 50862 and valuenum > 10 THEN null WHEN itemid = 50868 and valuenum > 10000 THEN null WHEN itemid = 51144 and valuenum < 0 THEN null WHEN itemid = 51144 and valuenum > 100 THEN null WHEN itemid = 50882 and valuenum > 10000 THEN null WHEN itemid = 50885 and valuenum > 150 THEN null WHEN itemid = 50806 and valuenum > 10000 THEN null WHEN itemid = 50902 and valuenum > 10000 THEN null WHEN itemid = 50912 and valuenum > 150 THEN null WHEN itemid = 50809 and valuenum > 10000 THEN null WHEN itemid = 50931 and valuenum > 10000 THEN null WHEN itemid = 50810 and valuenum > 100 THEN null WHEN itemid = 51221 and valuenum > 100 THEN null WHEN itemid = 50811 and valuenum > 50 THEN null WHEN itemid = 51222 and valuenum > 50 THEN null WHEN itemid = 50813 and valuenum > 50 THEN null WHEN itemid = 51265 and valuenum > 10000 THEN null WHEN itemid = 50822 and valuenum > 30 THEN null WHEN itemid = 50971 and valuenum > 30 THEN null WHEN itemid = 51275 and valuenum > 150 THEN null WHEN itemid = 51237 and valuenum > 50 THEN null WHEN itemid = 51274 and valuenum > 150 THEN null WHEN itemid = 50824 and valuenum > 200 THEN null WHEN itemid = 50983 and valuenum > 200 THEN null WHEN itemid = 51006 and valuenum > 300 THEN null WHEN itemid = 51300 and valuenum > 1000 THEN null WHEN itemid = 51301 and valuenum > 1000 THEN null ELSE le.valuenum END AS valuenum FROM mimiciii.icustays ie LEFT JOIN mimiciii.labevents le ON le.subject_id = ie.subject_id AND le.hadm_id = ie.hadm_id AND le.ITEMID in ( 50868, 50862, 51144, 50882, 50885, 50912, 50902, 50806, 50931, 50809, 51221, 50810, 51222, 50811, 50813, 51265, 50971, 50822, 51275, 51237, 51274, 50983, 50824, 51006, 51300, 51301, 50893, 51007, 50970, 50920 ) AND valuenum IS NOT null AND valuenum > 0) pvt GROUP BY pvt.subject_id, pvt.hadm_id, pvt.icustay_id, charttime"""

In [ ]:
# Connect to the PostgreSQL database with SQLAlchemy
import pandas as pd
from sqlalchemy import create_engine
import os
import psycopg2


host='172.26.121.29'
database='mimiciii'
user='mimic3_user'
password='x<FD-u{W$!3qu(tSCaco'
port='5432'

# Create a connection
engine = create_engine(f'postgresql://{user}:{password}@{host}:{port}/{database}')

# Define the queries
queries = {
    # "kdigo_stages_measured": "SELECT * FROM mimiciii.kdigo_stages WHERE icustay_id IN (SELECT icustay_id FROM mimiciii.kdigo_stages WHERE (creat IS NOT NULL OR uo_rt_6hr IS NOT NULL OR uo_rt_12hr IS NOT NULL OR uo_rt_24hr IS NOT NULL) AND aki_stage IS NOT NULL GROUP BY icustay_id HAVING COUNT(*) > 0)",
    # "icustay_detail": "SELECT * FROM mimiciii.icustay_detail WHERE icustay_id IN (SELECT icustay_id FROM mimiciii.kdigo_stages WHERE (creat IS NOT NULL OR uo_rt_6hr IS NOT NULL OR uo_rt_12hr IS NOT NULL OR uo_rt_24hr IS NOT NULL) AND aki_stage IS NOT NULL GROUP BY icustay_id HAVING COUNT(*) > 0)",
    # "vitals": "SELECT * FROM mimiciii.vitals WHERE icustay_id IN (SELECT icustay_id FROM mimiciii.kdigo_stages WHERE (creat IS NOT NULL OR uo_rt_6hr IS NOT NULL OR uo_rt_12hr IS NOT NULL OR uo_rt_24hr IS NOT NULL) AND aki_stage IS NOT NULL GROUP BY icustay_id HAVING COUNT(*) > 0)",
    "labs_original": "SELECT * FROM mimiciii.labs WHERE icustay_id IN (SELECT icustay_id FROM mimiciii.kdigo_stages WHERE (creat IS NOT NULL OR uo_rt_6hr IS NOT NULL OR uo_rt_12hr IS NOT NULL OR uo_rt_24hr IS NOT NULL) AND aki_stage IS NOT NULL GROUP BY icustay_id HAVING COUNT(*) > 0)",
    # "labs_extended" : labs_extended,
    # "labs_extended": f"SELECT * FROM ({labs_extended}) WHERE icustay_id IN (SELECT icustay_id FROM mimiciii.kdigo_stages WHERE (creat IS NOT NULL OR uo_rt_6hr IS NOT NULL OR uo_rt_12hr IS NOT NULL OR uo_rt_24hr IS NOT NULL) AND aki_stage IS NOT NULL GROUP BY icustay_id HAVING COUNT(*) > 0)",
    # "labs_extended": f"SELECT * FROM ({labs_extended}) AS subquery WHERE icustay_id IN (SELECT icustay_id FROM mimiciii.kdigo_stages WHERE (creat IS NOT NULL OR uo_rt_6hr IS NOT NULL OR uo_rt_12hr IS NOT NULL OR uo_rt_24hr IS NOT NULL) AND aki_stage IS NOT NULL GROUP BY icustay_id HAVING COUNT(*) > 0)",
    # "vents_vasopressor_sedatives": "SELECT ve.icustay_id AS icustay_id, ve.charttime AS charttime, vent, vasopressor, sedative FROM mimiciii.vent_kdigo_stages_labs_vitals_charttime ve, mimiciii.vasopressor_kdigo_stages_labs_vitals_charttime va, mimiciii.sedatives_kdigo_stages_labs_vitals_charttime s WHERE ve.icustay_id = va.icustay_id AND ve.charttime = va.charttime AND va.icustay_id = s.icustay_id AND va.charttime = s.charttime AND ve.icustay_id IN (SELECT icustay_id FROM mimiciii.kdigo_stages WHERE (creat IS NOT NULL OR uo_rt_6hr IS NOT NULL OR uo_rt_12hr IS NOT NULL OR uo_rt_24hr IS NOT NULL) AND aki_stage IS NOT NULL GROUP BY icustay_id HAVING COUNT(*) > 0) ORDER BY ve.icustay_id, ve.charttime",
    # "heightweight": "SELECT icustay_id, weight_first, height_first FROM mimiciii.heightweight WHERE icustay_id IN (SELECT icustay_id FROM mimiciii.kdigo_stages WHERE (creat IS NOT NULL OR uo_rt_6hr IS NOT NULL OR uo_rt_12hr IS NOT NULL OR uo_rt_24hr IS NOT NULL) AND aki_stage IS NOT NULL GROUP BY icustay_id HAVING COUNT(*) > 0)",
    # "calcium": "SELECT subject_id, hadm_id, icustay_id, charttime, calcium FROM mimiciii.blood_gas_first_day WHERE icustay_id IN (SELECT icustay_id FROM mimiciii.kdigo_stages WHERE (creat IS NOT NULL OR uo_rt_6hr IS NOT NULL OR uo_rt_12hr IS NOT NULL OR uo_rt_24hr IS NOT NULL) AND aki_stage IS NOT NULL GROUP BY icustay_id HAVING COUNT(*) > 0)",
    # "inr_max": "SELECT subject_id, hadm_id, icustay_id, inr_max FROM mimiciii.meld WHERE icustay_id IN (SELECT icustay_id FROM mimiciii.kdigo_stages WHERE (creat IS NOT NULL OR uo_rt_6hr IS NOT NULL OR uo_rt_12hr IS NOT NULL OR uo_rt_24hr IS NOT NULL) AND aki_stage IS NOT NULL GROUP BY icustay_id HAVING COUNT(*) > 0)"
}

# Execute the queries and write the results to CSV files
for name, query in queries.items():
    df = pd.read_sql_query(query, engine)
    out_dir = '../data/extracted'
    os.makedirs(out_dir, exist_ok=True)
    df.to_csv(os.path.join(out_dir,f'{name}.csv'), index=False, sep=';')